In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
#from seaborn.linearmodels import corrplot,symmatplot
        
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error, r2_score

In [52]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

DATA_DIR = '../data/'
df_population = pd.read_csv(DATA_DIR + 'df_population_census.csv')

#df_growth = df_population.groupby(['Year']).sum()
#pop2016 = (df_growth['Year'] == 2016).sum()
pop2000 = df_population.loc[df_population['Year'] == 2000, 'Population'].sum()
pop2010 = df_population.loc[df_population['Year'] == 2010, 'Population'].sum()
pop2016 = df_population.loc[df_population['Year'] == 2016, 'Population'].sum()

growth2016 = pop2016/pop2010
growth2010 = pop2010/pop2000
print (growth2016,growth2010)

df_2016 = df_population[df_population['Year'] == 2016]

Year = []
Population = []
Block_Group = []
Population_x = []

for index, row in df_2016.iterrows():
    
    Year.append(2010)
    Block_Group.append(row['Block_Group'])
    Population_x.append(row['Population'])
    Population.append(df_population.loc[(df_population['Year'] == 2010) & 
                (df_population['Block_Group'] == row['Block_Group']), 'Population'].sum())
    
# create new dataframe for 2016
df_2010 = pd.DataFrame()
df_2010['Block_Group'] = Block_Group
df_2010['Population'] = Population
df_2010['Year'] = Year
df_2010['Population_x'] = Population_x
df_2010['Population'] = np.where(df_2010['Population'] == 0, df_2010['Population_x']/growth2016, df_2010['Population'])
df_2010['Population'] = df_2010['Population'] .astype(int)
df_2010 = df_2010.drop(columns=['Population_x'])


Year = []
Population = []
Block_Group = []
Population_x = []

for index, row in df_2010.iterrows():
    
    Year.append(2000)
    Block_Group.append(row['Block_Group'])
    Population_x.append(row['Population'])
    Population.append(df_population.loc[(df_population['Year'] == 2000) & 
                (df_population['Block_Group'] == row['Block_Group']), 'Population'].sum())
    
# create new dataframe for 2000
df_2000 = pd.DataFrame()
df_2000['Block_Group'] = Block_Group
df_2000['Population'] = Population
df_2000['Year'] = Year
df_2000['Population_x'] = Population_x
df_2000['Population'] = np.where(df_2000['Population'] == 0, df_2000['Population_x']/growth2010, df_2000['Population'])
df_2000['Population'] = df_2000['Population'] .astype(int)

df_2000 = df_2000.drop(columns=['Population_x'])

print (df_2016.head())
print (df_2010.head())
print (df_2000.head())
df_population = pd.concat([df_2016,df_2010,df_2000])

# Adjust ACS totals to match official census population
df_population['Population'] = np.where(df_population['Year'] < 2010, 
        df_population['Population'] * 1.09, df_population['Population'])

df_population['Population'] = np.where(df_population['Year'] == 2010, 
        df_population['Population'] * .9940, df_population['Population'])

df_population['Population'] = np.where(df_population['Year'] == 2016, 
        df_population['Population'] * 1.0203, df_population['Population'])

print (df_population.head())



# If there is no year, calculate the average year

1.0767902015742143 1.1118084044411336
       Block_Group  Population  Year
3002  530330001001        1248  2016
3003  530330001002        1336  2016
3004  530330001003        1785  2016
3005  530330001004        1953  2016
3006  530330001005         939  2016
    Block_Group  Population  Year
0  530330001001        1250  2010
1  530330001002        1234  2010
2  530330001003        1337  2010
3  530330001004        1492  2010
4  530330001005         942  2010
    Block_Group  Population  Year
0  530330001001        1288  2000
1  530330001002        1327  2000
2  530330001003        1548  2000
3  530330001004        1367  2000
4  530330001005         847  2000
       Block_Group  Population  Year
3002  530330001001   1273.3344  2016
3003  530330001002   1363.1208  2016
3004  530330001003   1821.2355  2016
3005  530330001004   1992.6459  2016
3006  530330001005    958.0617  2016


In [53]:
Years = []
for i in range(21):
    Years.append(int(2000 + i))

df_years = pd.DataFrame({'Year':Years})
#print (df_years)


In [54]:
Geographies = set(df_population["Block_Group"].tolist())

Block_Group = []
Population = []
Year = []
MSE = []
Status = []

df_predictions_full = pd.DataFrame()

for geography in Geographies:
    
    df_data = df_population[df_population["Block_Group"] == geography]
    #print (df_data)
    
    for index, row in df_data.iterrows():
        
        Block_Group.append(geography)
        Year.append(row["Year"]) 
        Population.append(row["Population"]) 
        Status.append("actual")
            
        if row["Year"] == 2016:
            
            X = df_data[["Year"]]
            y = df_data['Population']

            # Instantiate the model
            #model = LinearRegression()
            model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                              ('linear', LinearRegression(fit_intercept=False))])

            # Fit model to features and target
            model.fit(X,y)
            predictionsX = model.predict(X)
            mse = mean_squared_error(y, predictionsX)
                
            # Make predictions
            pred = df_years[["Year"]]
            predictions = model.predict(df_years)

            for i, name in enumerate(predictions):
            
                Block_Group.append(geography)
                Year.append(Years[i]) 
                Population.append(predictions[i]) 
                Status.append("prediction")

df_predictions_full['Block_Group'] = Block_Group
df_predictions_full['Year'] = Year
df_predictions_full['Population'] = Population
df_predictions_full['Status'] = Status

# Manually fudge predictions to match State estimates

#2011 	0.9757	0.9722	0.9735	0.9857	0.9999	1.0161	1.0163


df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2010, 
        df_predictions_full['Population'] * 1.000, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2011, 
        df_predictions_full['Population'] * 0.9874, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2012, 
        df_predictions_full['Population'] * 0.9757, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2013, 
        df_predictions_full['Population'] * 0.9722, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2014, 
        df_predictions_full['Population'] * 0.9735, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2015, 
        df_predictions_full['Population'] * 0.9857, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2016, 
        df_predictions_full['Population'] * 1.0, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2017, 
        df_predictions_full['Population'] * 1.0161, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2018, 
        df_predictions_full['Population'] * 1.0163, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2019, 
        df_predictions_full['Population'] * 1.0163, df_predictions_full['Population'])

df_predictions_full['Population'] = np.where(df_predictions_full['Year'] == 2020, 
        df_predictions_full['Population'] * 1.0163, df_predictions_full['Population'])

df_predictions_full['Population'] = df_predictions_full['Population'] .astype(int)


#print (df_predictions_full)

df_predictions_full.to_csv(DATA_DIR + 'Population_Predictions.csv', mode='w', header=True, index=False)

pop2000 = df_predictions_full.loc[(df_predictions_full['Year'] == 2000) & 
                                  (df_predictions_full['Status'] == "prediction"), 'Population'].sum()
pop2018 = df_predictions_full.loc[(df_predictions_full['Year'] == 2018) & 
                                  (df_predictions_full['Status'] == "prediction"), 'Population'].sum()
print (pop2018/pop2000)



# Calulate mse
#mse = mean_squared_error(y, predictions)
#variance = r2_score(y, predictions)

#df_population['predictions'] = predictions
#print (df_population)


1.2608575235835278


In [55]:
df_predictions_full.to_csv(DATA_DIR + 'Population_Predictions.csv', mode='w', header=True, index=False)


In [56]:
print (df_predictions_full)

        Block_Group    Year  Population      Status
0      530330026001  2016.0        1223      actual
1      530330026001  2000.0        1120  prediction
2      530330026001  2001.0        1092  prediction
3      530330026001  2002.0        1069  prediction
4      530330026001  2003.0        1050  prediction
5      530330026001  2004.0        1036  prediction
6      530330026001  2005.0        1027  prediction
7      530330026001  2006.0        1022  prediction
8      530330026001  2007.0        1021  prediction
9      530330026001  2008.0        1025  prediction
10     530330026001  2009.0        1034  prediction
11     530330026001  2010.0        1047  prediction
12     530330026001  2011.0        1052  prediction
13     530330026001  2012.0        1061  prediction
14     530330026001  2013.0        1083  prediction
15     530330026001  2014.0        1115  prediction
16     530330026001  2015.0        1165  prediction
17     530330026001  2016.0        1223  prediction
18     53033